# Modal Sense Classification using LSTM

In [ ]:
# Python version
from platform import python_version
print(python_version())

3.8.15


## Loading Dataset

In [ ]:
# Loading packages for Data Manipulation
import pandas as pd
import os
import glob
import numpy as np

In [ ]:
# Dataset folder
dataset_folder = "data/MASC"
# List to store dataset (txt files) inside the folder as pandas dataframe
df_files = []

# Creating a list of pandas dataframe by going through all the text file with column name 'text'& 'modality'
for file in glob.glob(os.path.join(dataset_folder,"*.txt")):
  print(file + "...Done...")
  df_files.append(pd.read_csv(file, delimiter='\t', quoting=3, names =['text', 'modality']))

data/MASC/travel-guides__should.txt...Done...
data/MASC/fiction__can.txt...Done...
data/MASC/movie-script__could.txt...Done...
data/MASC/jokes__shall.txt...Done...
data/MASC/journal__shall.txt...Done...
data/MASC/telephone__can.txt...Done...
data/MASC/debate-transcript__may.txt...Done...
data/MASC/ficlets__can.txt...Done...
data/MASC/fiction__could.txt...Done...
data/MASC/non-fiction__may.txt...Done...
data/MASC/travel-guides__must.txt...Done...
data/MASC/essays__could.txt...Done...
data/MASC/court-transcript__may.txt...Done...
data/MASC/travel-guides__can.txt...Done...
data/MASC/blog__can.txt...Done...
data/MASC/blog__could.txt...Done...
data/MASC/technical__shall.txt...Done...
data/MASC/non-fiction__shall.txt...Done...
data/MASC/newspaper__should.txt...Done...
data/MASC/movie-script__can.txt...Done...
data/MASC/essays__may.txt...Done...
data/MASC/govt-docs__shall.txt...Done...
data/MASC/fiction__must.txt...Done...
data/MASC/telephone__shall.txt...Done...
data/MASC/journal__must.txt..

In [ ]:
# Concatenating the list of dataframes into one
df = pd.concat(df_files,ignore_index=True)
df.head()

,text,modality
0,"Dublin theater is legendary , and no visitor s...",de
1,The large department stores have fixed prices ...,de
2,"Before purchasing , visitors should make sure ...",de
3,You should always ask for a certificate of gua...,de
4,"However , you are not likely to find bargains ...",de


In [ ]:
# dropping any other modalities except 'ep', 'de' & 'dy'
df = df.loc[df['modality'].isin(['ep', 'de', 'dy'])].reset_index(drop=True)

In [ ]:
# removing duplicate entries
df = df.drop_duplicates(ignore_index = True)

In [ ]:
len(df['text'])

1847

In [ ]:
# annotating modal verbs for individual text
mverb_list = []
for i in range(len(df['text'])):
  text = str(df['text'][i]).lower()
  if text.find('can') != -1:
    mverb = 'can'
  elif text.find('could') != -1:
    mverb = 'could'
  elif text.find('may') != -1:
    mverb = 'may'
  elif text.find('must') != -1:
    mverb = 'must'
  elif text.find('shall') != -1:
    mverb = 'shall'
  elif text.find('should') != -1:
    mverb = 'should'
  mverb_list.append(mverb)

In [ ]:
len(mverb_list)

1847

In [ ]:
df['modal_verb'] = mverb_list

In [ ]:
print(df.modal_verb.unique())

['should' 'can' 'could' 'may' 'must' 'shall']


In [ ]:
df.shape

(1847, 3)

In [ ]:
df

,text,modality,modal_verb
0,"Dublin theater is legendary , and no visitor s...",de,should
1,The large department stores have fixed prices ...,de,should
2,"Before purchasing , visitors should make sure ...",de,should
3,You should always ask for a certificate of gua...,de,should
4,"However , you are not likely to find bargains ...",de,can
...,...,...,...
1842,"However , because the stream can adjust , the ...",dy,can
1843,The probability that a large river can adjust ...,dy,can
1844,Changes of valley - floor gradient provide an ...,ep,can
1845,"For example , deposition upstream from the axi...",dy,can


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1847 entries, 0 to 1846
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        1847 non-null   object
 1   modality    1847 non-null   object
 2   modal_verb  1847 non-null   object
dtypes: object(3)
memory usage: 43.4+ KB


In [ ]:
# value counts of individual modality according to the modal verbs
pd.crosstab(df['modality'], df['modal_verb'])

modal_verb,can,could,may,must,shall,should
modality,,,,,,
de,103,16,44,98,1,194
dy,671,235,4,0,1,2
ep,103,129,194,27,0,25


## Data Preprocessing

In [ ]:
# keeping only 'ep' and 'de' modalities
df_final = df.loc[df['modality'].isin(['ep', 'de'])].reset_index(drop=True)

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        934 non-null    object
 1   modality    934 non-null    object
 2   modal_verb  934 non-null    object
dtypes: object(3)
memory usage: 22.0+ KB


In [ ]:
df_final.modality.value_counts()

ep    478
de    456
Name: modality, dtype: int64

In [ ]:
df_final.head()

,text,modality,modal_verb
0,"Dublin theater is legendary , and no visitor s...",de,should
1,The large department stores have fixed prices ...,de,should
2,"Before purchasing , visitors should make sure ...",de,should
3,You should always ask for a certificate of gua...,de,should
4,"However , you are not likely to find bargains ...",de,can


## Train-Test Split

In [ ]:
# Loading sklearn for train-test split
from sklearn.model_selection import train_test_split

In [ ]:
# Converting the columns to list
x = df_final['text'].tolist()
y = df_final['modality'].tolist()

In [ ]:
# Spliting the data for 70% train data, 15% validation data, and 15% test data 
x_train, x_rem, y_train, y_rem = train_test_split(x, y, train_size=0.7, shuffle=True)
x_val, x_test, y_val, y_test = train_test_split(x_rem, y_rem, test_size=0.5, shuffle=True)

In [ ]:
# Checking the sizes of each split
print("Train Set: ",len(x_train), len(y_train))
print("Validation Set: ",len(x_val), len(y_val))
print("Test Set: ",len(x_test), len(y_test))

Train Set:  653 653
Validation Set:  140 140
Test Set:  141 141


In [ ]:
# Checking the maximum length of the sentences
max_train = max([len(s.split()) for s in x_train])
max_test = max([len(s.split()) for s in x_test])
max_val = max([len(s.split()) for s in x_val])

In [ ]:
print("Max train length", max_train)
print("Max test length", max_test)
print("Max validation length", max_val)

Max train length 76
Max test length 80
Max validation length 74


## Padding

In [ ]:
# Loading package for padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Padding to have same length according to the maximum sentence length
def pad_text(embeddedSents):
  padded = pad_sequences(embeddedSents, padding= 'post', dtype='float32', truncating = 'post', maxlen = 80)
  return padded

## Label Encoding

In [ ]:
# Loading packages for One-Hot Encoding
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from numpy import array

In [ ]:
# Encode Labels into Categorical One-Hot Vectors
def encode_y(labels):
  label_encoder = LabelEncoder()
  # transform string labels into integers
  y_encoded = label_encoder.fit_transform(labels) 
  return y_encoded

In [ ]:
# Converting to binary encoding
y_e_train = encode_y(y_train)
y_e_val = encode_y(y_val)
y_e_test = encode_y(y_test)

In [ ]:
y_e_train.shape

(653,)

## Word Embeddings

In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Loading packages for Word Embeddings
from flair.data import Sentence
from flair.embeddings import TransformerWordEmbeddings

In [ ]:
# General method to convert into different word embedding techniques
def embed_text(text, embeddings):
  embeddedSents = []
  for line in text:
    sent = Sentence(line)
    # transform each sentence into word vector sequence
    embeddings.embed(sent) 
    embeddedSents.append([t.embedding.cpu().numpy() for t in sent])
  return embeddedSents

### XLNet

In [ ]:
# init XLNet embedding
em_xlnet = TransformerWordEmbeddings('xlnet-large-cased') # this might take a long time for the first time

In [ ]:
# Embedding each set. This will take some time.
x_train_xlnet = embed_text(x_train, em_xlnet)
x_val_xlnet = embed_text(x_val, em_xlnet)
x_test_xlnet = embed_text(x_test, em_xlnet)

#### Padding for XLNet 

In [ ]:
# Padding each set
x_ftr_xlnet = pad_text(x_train_xlnet)
x_fv_xlnet = pad_text(x_val_xlnet)
x_fts_xlnet = pad_text(x_test_xlnet)

In [ ]:
x_ftr_xlnet.shape

(653, 80, 1024)

#### LSTM with XLNet

In [ ]:
# Loading packages for LSTM model
import tensorflow as tf
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

In [ ]:
# Model initialization
max_features = 50000
dim = 1024
size = 80

model_xlnet = tf.keras.Sequential()
#model_xlnet.add(tf.keras.layers.Embedding(max_features, dim, input_length = size))
#model_xlnet.add(tf.keras.layers.Dropout(0.4))

model_xlnet.add(tf.keras.layers.LSTM(dim, input_shape=(80,1024), dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model_xlnet.add(tf.keras.layers.Flatten())

model_xlnet.add(tf.keras.layers.Dense(512, activation='relu'))
model_xlnet.add(tf.keras.layers.Dropout(0.4))

model_xlnet.add(tf.keras.layers.Dense(128, activation='relu'))
model_xlnet.add(tf.keras.layers.Dropout(0.4))

model_xlnet.add(tf.keras.layers.Dense(32, activation='relu'))
model_xlnet.add(tf.keras.layers.Dropout(0.4))

model_xlnet.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model_xlnet.summary()
model_xlnet.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.legacy.Adam(1e-3),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 80, 1024)          8392704   
                                                                 
 flatten (Flatten)           (None, 81920)             0         
                                                                 
 dense (Dense)               (None, 512)               41943552  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 32)                4

In [ ]:
# Training the model
history_xlnet = model_xlnet.fit(x = x_ftr_xlnet, y = y_e_train, 
                          validation_data = (x_fv_xlnet, y_e_val), 
                          shuffle = True, epochs = 20, batch_size = 128)

Epoch 1/20
6/6 [==============================] - 6s 554ms/step - loss: 2.1715 - binary_accuracy: 0.4946 - val_loss: 0.7060 - val_binary_accuracy: 0.5000
Epoch 2/20
6/6 [==============================] - 4s 664ms/step - loss: 0.9018 - binary_accuracy: 0.5253 - val_loss: 0.7103 - val_binary_accuracy: 0.4714
Epoch 3/20
6/6 [==============================] - 4s 584ms/step - loss: 0.8379 - binary_accuracy: 0.5237 - val_loss: 0.6975 - val_binary_accuracy: 0.5714
Epoch 4/20
6/6 [==============================] - 3s 431ms/step - loss: 0.7574 - binary_accuracy: 0.5406 - val_loss: 0.6929 - val_binary_accuracy: 0.5357
Epoch 5/20
6/6 [==============================] - 3s 426ms/step - loss: 0.7379 - binary_accuracy: 0.5008 - val_loss: 0.6989 - val_binary_accuracy: 0.4214
Epoch 6/20
6/6 [==============================] - 3s 424ms/step - loss: 0.7125 - binary_accuracy: 0.5253 - val_loss: 0.6950 - val_binary_accuracy: 0.4786
Epoch 7/20
6/6 [==============================] - 3s 430ms/step - loss: 0.69

In [ ]:
# Saving the model
model_xlnet.save('lstm_xlnet.h5') 

In [ ]:
# Loading the model
#loaded_model_xlnet = tf.keras.models.load_model('lstm_xlnet.h5')

In [ ]:
# Evaluating the model
results_xlnet = model_xlnet.evaluate(x_fts_xlnet,y_e_test)

5/5 [==============================] - 0s 75ms/step - loss: 0.7621 - binary_accuracy: 0.7376


### RoBERTa

In [ ]:
# init RoBERTa embedding
em_rba = TransformerWordEmbeddings('roberta-large') # this might take a long time for the first time

In [ ]:
# Embedding each set. This will take some time.
x_train_rba = embed_text(x_train, em_rba)
x_val_rba = embed_text(x_val, em_rba)
x_test_rba = embed_text(x_test, em_rba)

#### Padding for RoBERTa

In [ ]:
# Padding each set
x_ftr_rba = pad_text(x_train_rba)
x_fv_rba = pad_text(x_val_rba)
x_fts_rba = pad_text(x_test_rba)

In [ ]:
x_ftr_rba.shape

(653, 80, 1024)

#### LSTM with RoBERTa

In [ ]:
# Loading packages for LSTM model
import tensorflow as tf
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

# Model initialization
max_features = 50000
dim = 1024
size = 80

model_rba = tf.keras.Sequential()
#model_rba.add(tf.keras.layers.Embedding(max_features, dim, input_length = size))
#model_rba.add(tf.keras.layers.Dropout(0.4))

model_rba.add(tf.keras.layers.LSTM(dim, input_shape=(80,1024), dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model_rba.add(tf.keras.layers.Flatten())

model_rba.add(tf.keras.layers.Dense(512, activation='relu'))
model_rba.add(tf.keras.layers.Dropout(0.4))

model_rba.add(tf.keras.layers.Dense(128, activation='relu'))
model_rba.add(tf.keras.layers.Dropout(0.4))

model_rba.add(tf.keras.layers.Dense(32, activation='relu'))
model_rba.add(tf.keras.layers.Dropout(0.4))

model_rba.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model_rba.summary()
model_rba.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                  optimizer=tf.keras.optimizers.legacy.Adam(1e-3),
                  metrics=[tf.keras.metrics.BinaryAccuracy()])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 80, 1024)          8392704   
                                                                 
 flatten_1 (Flatten)         (None, 81920)             0         
                                                                 
 dense_4 (Dense)             (None, 512)               41943552  
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               65664     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 32)               

In [ ]:
# Training the model
history_rba = model_rba.fit(x = x_ftr_rba, y = y_e_train, 
                          validation_data = (x_fv_rba, y_e_val), 
                          shuffle = True, epochs = 20, batch_size = 128)

Epoch 1/20
6/6 [==============================] - 5s 502ms/step - loss: 2.7762 - binary_accuracy: 0.5023 - val_loss: 1.4436 - val_binary_accuracy: 0.3786
Epoch 2/20
6/6 [==============================] - 3s 430ms/step - loss: 1.0428 - binary_accuracy: 0.5084 - val_loss: 0.6818 - val_binary_accuracy: 0.6143
Epoch 3/20
6/6 [==============================] - 3s 437ms/step - loss: 0.7608 - binary_accuracy: 0.5176 - val_loss: 0.6753 - val_binary_accuracy: 0.6143
Epoch 4/20
6/6 [==============================] - 3s 452ms/step - loss: 0.7434 - binary_accuracy: 0.5375 - val_loss: 0.6753 - val_binary_accuracy: 0.6071
Epoch 5/20
6/6 [==============================] - 3s 435ms/step - loss: 0.6966 - binary_accuracy: 0.5773 - val_loss: 0.6801 - val_binary_accuracy: 0.6286
Epoch 6/20
6/6 [==============================] - 3s 432ms/step - loss: 0.6807 - binary_accuracy: 0.6080 - val_loss: 0.6385 - val_binary_accuracy: 0.6786
Epoch 7/20
6/6 [==============================] - 3s 437ms/step - loss: 0.65

In [ ]:
# Saving the model
model_rba.save('lstm_roberta.h5') 

In [ ]:
# Loading the model
#loaded_model_rba = tf.keras.models.load_model('lstm_roberta.h5')

In [ ]:
# Evaluating the model
results_rba = model_rba.evaluate(x_fts_rba,y_e_test)

5/5 [==============================] - 0s 75ms/step - loss: 0.4722 - binary_accuracy: 0.8227


## Cross Validation

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)

### Data Preperation

In [ ]:
# Converting whole dataset to One-Hot encoding
label_encoder = LabelEncoder()
y_final = label_encoder.fit_transform(y)

In [ ]:
y_final.shape

(934,)

### Cross Validation with XLNet

In [ ]:
# Converting whole dataset with Word Embeddings
# This might take some time
x_xlnet = embed_text(x, em_xlnet)

In [ ]:
# Padding for whole dataset
x_pad_xlnet = pad_text(x_xlnet)

In [ ]:
x_pad_xlnet.shape

(934, 80, 1024)

In [ ]:
# Loading packages for LSTM model
import tensorflow as tf
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

In [ ]:
# List to store cv scores
cvscores_xlnet = []
# Model initialization
max_features = 50000
dim = 1024
size = 80

for train, test in kfold.split(x_pad_xlnet, y_final):
  model_xlnet = tf.keras.Sequential()
  #model_xlnet.add(tf.keras.layers.Embedding(max_features, dim, input_length = size))
  #model_xlnet.add(tf.keras.layers.Dropout(0.4))

  model_xlnet.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(dim, dropout=0.2, recurrent_dropout=0.2, 
                                                                     return_sequences=True),input_shape=(80,1024)))
  model_xlnet.add(tf.keras.layers.Flatten())

  model_xlnet.add(tf.keras.layers.Dense(512, activation='relu'))
  model_xlnet.add(tf.keras.layers.Dropout(0.4))

  model_xlnet.add(tf.keras.layers.Dense(128, activation='relu'))
  model_xlnet.add(tf.keras.layers.Dropout(0.4))

  model_xlnet.add(tf.keras.layers.Dense(32, activation='relu'))
  model_xlnet.add(tf.keras.layers.Dropout(0.4))

  model_xlnet.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  model_xlnet.summary()
  model_xlnet.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                      optimizer=tf.keras.optimizers.legacy.Adam(1e-3),
                      metrics=[tf.keras.metrics.BinaryAccuracy()])
  
  # Training the model
  model_xlnet.fit(x_pad_xlnet[train], y_final[train], epochs = 20, batch_size = 128)
  # Evaluating the model
  scores_xlnet = model_xlnet.evaluate(x_pad_xlnet[test], y_final[test], verbose=0)
  print("Model Accuracy: ", scores_xlnet[1]*100)
  cvscores_xlnet.append(scores_xlnet[1] * 100)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 80, 2048)         16785408  
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 163840)            0         
                                                                 
 dense (Dense)               (None, 512)               83886592  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                        

Model Accuracy:  81.81818127632141
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirectio  (None, 80, 2048)         16785408  
 nal)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 163840)            0         
                                                                 
 dense_4 (Dense)             (None, 512)               83886592  
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               65664     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                   

Model Accuracy:  73.79679083824158
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 80, 2048)         16785408  
 nal)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 163840)            0         
                                                                 
 dense_8 (Dense)             (None, 512)               83886592  
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 128)               65664     
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                   

Model Accuracy:  73.79679083824158
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_3 (Bidirectio  (None, 80, 2048)         16785408  
 nal)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 163840)            0         
                                                                 
 dense_12 (Dense)            (None, 512)               83886592  
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 128)               65664     
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                   

Model Accuracy:  71.65775299072266
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_4 (Bidirectio  (None, 80, 2048)         16785408  
 nal)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 163840)            0         
                                                                 
 dense_16 (Dense)            (None, 512)               83886592  
                                                                 
 dropout_12 (Dropout)        (None, 512)               0         
                                                                 
 dense_17 (Dense)            (None, 128)               65664     
                                                                 
 dropout_13 (Dropout)        (None, 128)               0         
                   

In [ ]:
print("CV Scores: ", cvscores_xlnet)
print("Mean Accuracy: ", np.mean(cvscores_xlnet))
print("Standard Deviation: ", np.std(cvscores_xlnet))

CV Scores:  [81.81818127632141, 73.79679083824158, 73.79679083824158, 71.65775299072266, 70.43010592460632]
Mean Accuracy:  74.29992437362671
Standard Deviation:  3.974595275214584


### Cross Validation with RoBERTa

In [ ]:
# Converting whole dataset with Word Embeddings
# This might take some time
x_rba = embed_text(x, em_rba)

In [ ]:
x_pad_rba = pad_text(x_rba)

In [ ]:
x_pad_rba.shape

(934, 80, 1024)

In [ ]:
# Loading packages for LSTM model
import tensorflow as tf
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

In [ ]:
# List to store cv scores
cvscores_rba = []
# Model initialization
max_features = 50000
dim = 1024
size = 80

for train, test in kfold.split(x_pad_rba, y_final):
  model_rba = tf.keras.Sequential()
  #model_xlnet.add(tf.keras.layers.Embedding(max_features, dim, input_length = size))
  #model_xlnet.add(tf.keras.layers.Dropout(0.4))

  model_rba.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(dim, dropout=0.2, recurrent_dropout=0.2, 
                                                                   return_sequences=True),input_shape=(80,1024)))
  model_rba.add(tf.keras.layers.Flatten())

  model_rba.add(tf.keras.layers.Dense(512, activation='relu'))
  model_rba.add(tf.keras.layers.Dropout(0.4))

  model_rba.add(tf.keras.layers.Dense(128, activation='relu'))
  model_rba.add(tf.keras.layers.Dropout(0.4))

  model_rba.add(tf.keras.layers.Dense(32, activation='relu'))
  model_rba.add(tf.keras.layers.Dropout(0.4))

  model_rba.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  model_rba.summary()
  model_rba.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                      optimizer=tf.keras.optimizers.legacy.Adam(1e-3),
                      metrics=[tf.keras.metrics.BinaryAccuracy()])
  
  # Training the model
  model_rba.fit(x_pad_rba[train], y_final[train], epochs = 20, batch_size = 128)
  # Evaluating the model
  scores_rba = model_rba.evaluate(x_pad_rba[test], y_final[test], verbose=0)
  print("Model Accuracy: ", scores_rba[1]*100)
  cvscores_rba.append(scores_rba[1] * 100)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 80, 2048)         16785408  
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 163840)            0         
                                                                 
 dense (Dense)               (None, 512)               83886592  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                        

Model Accuracy:  82.88770318031311
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirectio  (None, 80, 2048)         16785408  
 nal)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 163840)            0         
                                                                 
 dense_4 (Dense)             (None, 512)               83886592  
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               65664     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                   

Model Accuracy:  88.77005577087402
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 80, 2048)         16785408  
 nal)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 163840)            0         
                                                                 
 dense_8 (Dense)             (None, 512)               83886592  
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 128)               65664     
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                   

Model Accuracy:  86.6310179233551
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_3 (Bidirectio  (None, 80, 2048)         16785408  
 nal)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 163840)            0         
                                                                 
 dense_12 (Dense)            (None, 512)               83886592  
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 128)               65664     
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                    

Model Accuracy:  83.42245817184448
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_4 (Bidirectio  (None, 80, 2048)         16785408  
 nal)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 163840)            0         
                                                                 
 dense_16 (Dense)            (None, 512)               83886592  
                                                                 
 dropout_12 (Dropout)        (None, 512)               0         
                                                                 
 dense_17 (Dense)            (None, 128)               65664     
                                                                 
 dropout_13 (Dropout)        (None, 128)               0         
                   

In [ ]:
print("CV Scores: ", cvscores_rba)
print("Mean Accuracy: ", np.mean(cvscores_rba))
print("Standard Deviation: ", np.std(cvscores_rba))

CV Scores:  [82.88770318031311, 88.77005577087402, 86.6310179233551, 83.42245817184448, 84.40860509872437]
Mean Accuracy:  85.22396802902222
Standard Deviation:  2.187368994704828
